In [53]:
import pandas as pd
import numpy as np
import tensorflow as tf
import math
import os
import sys
from datetime import datetime
from sklearn.model_selection import train_test_split

In [54]:
cur = os.getcwd()
log_dir = os.path.join(cur,'logs')
print(log_dir)

/Users/EstherChang/Documents/Version_Control/Tensorboard/logs


In [55]:
titanic = pd.read_csv("titanic.csv")

In [56]:
titanic.head()

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500


In [57]:
label = titanic['Survived']
data = titanic.drop(['Survived','Name'], axis=1)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['Sex'] = le.fit_transform(data['Sex'])
print(data.shape)

(887, 6)


In [58]:
data = np.asarray(data).astype('float32')
label = np.asarray(label).astype('float32').reshape((-1,1))
x_train, x_test, y_train, y_test = train_test_split(data, label, test_size = 0.2)

In [59]:
tf.reset_default_graph()

In [60]:
sess = tf.InteractiveSession()

In [61]:
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 6], name = 'x-input')
    y_ = tf.placeholder(tf.float32, [None,1], name = 'y-input')

In [62]:
def weight_variable(shape):
    weight = tf.Variable(tf.truncated_normal(shape, stddev = 0.1))
    return weight
def bias_variable(shape):
    bias = tf.Variable(tf.constant(0.1, shape=shape))
    return bias
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('hist', var)

In [63]:
def nn_layer(input_tensor, input_dim, output_dim, layer_name, activation=tf.nn.relu):
    with tf.name_scope(layer_name):
        with tf.name_scope('weight'):
            weight = weight_variable([input_dim, output_dim])
            variable_summaries(weight)
        with tf.name_scope('bias'):
            bias = bias_variable([output_dim])
            variable_summaries(bias)
        with tf.name_scope('Wx_plus_b'):
            preactivate = tf.matmul(input_tensor, weight) + bias
            tf.summary.histogram('preactivate', preactivate)
        
        activations = activation(preactivate, name = 'activation')
        tf.summary.histogram('activation', activations)
    return activations

In [64]:
hidden1 = nn_layer(x, 6, 15, 'layer1')

with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    tf.summary.scalar('keep_prob', keep_prob)
    dropped = tf.nn.dropout(hidden1, keep_prob)

y = nn_layer(dropped, 15, 1, 'layer2', activation=tf.identity)

In [65]:
with tf.name_scope('cross_entropy'):
    with tf.name_scope('total'):
        cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels = y_, logits = y)
        loss = tf.reduce_mean(cross_entropy)
    tf.summary.scalar('loss', loss)

In [66]:
learning_rate = 0.005
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        pred = tf.round(tf.nn.sigmoid(y))
        correct_pred = tf.equal(pred, y_)
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
tf.summary.scalar('accuracy', accuracy)
tf.summary.histogram('correct_pred', tf.cast(correct_pred, tf.int64))
from tensorboard import summary as summary_lib
from tensorboard.plugins.pr_curve import summary
#summary_proto = summary_lib.pr_curve(name = "pr", predictions = pred, labels = tf.cast(y_, tf.bool))
#_, update_op = summary_lib.pr_curve_streaming_op(name='pr',
#                                                 predictions=pred,
#                                                 labels=tf.cast(y_, tf.bool),
#                                                 num_thresholds=11)
summary.op(name='pr', predictions=tf.nn.sigmoid(y), labels=tf.cast(y_, tf.bool), num_thresholds=100)

<tf.Tensor 'pr/pr_curves:0' shape=() dtype=string>

In [67]:
merged_summary = tf.summary.merge_all()
time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
train_writer = tf.summary.FileWriter(log_dir + '/train' + time, sess.graph)
test_writer = tf.summary.FileWriter(log_dir + '/test' + time)
tf.global_variables_initializer().run()
tf.local_variables_initializer().run()

In [68]:
epoch = 500
for i in range(epoch):
    if i % 10 == 0:
        summary, acc = sess.run([merged_summary, accuracy], 
                                feed_dict = {x:x_test, y_: y_test, keep_prob:1.0})
        test_writer.add_summary(summary, i)
        print('Accuracy at step %s: %s' %(i, acc))
    else:
        summary, _ = sess.run([merged_summary, train_step], 
                              feed_dict = {x:x_train, y_: y_train, keep_prob:0.8} )
        train_writer.add_summary(summary, i)
train_writer.close()
test_writer.close()
sess.close()

Accuracy at step 0: 0.7022472
Accuracy at step 10: 0.6966292
Accuracy at step 20: 0.6797753
Accuracy at step 30: 0.66853935
Accuracy at step 40: 0.6797753
Accuracy at step 50: 0.6853933
Accuracy at step 60: 0.69101125
Accuracy at step 70: 0.6797753
Accuracy at step 80: 0.66853935
Accuracy at step 90: 0.7022472
Accuracy at step 100: 0.6966292
Accuracy at step 110: 0.6853933
Accuracy at step 120: 0.69101125
Accuracy at step 130: 0.7078652
Accuracy at step 140: 0.75842696
Accuracy at step 150: 0.747191
Accuracy at step 160: 0.747191
Accuracy at step 170: 0.7078652
Accuracy at step 180: 0.752809
Accuracy at step 190: 0.7303371
Accuracy at step 200: 0.74157304
Accuracy at step 210: 0.74157304
Accuracy at step 220: 0.71910113
Accuracy at step 230: 0.74157304
Accuracy at step 240: 0.74157304
Accuracy at step 250: 0.74157304
Accuracy at step 260: 0.752809
Accuracy at step 270: 0.73595506
Accuracy at step 280: 0.747191
Accuracy at step 290: 0.74157304
Accuracy at step 300: 0.752809
Accuracy at 